In [2]:
import cv2
import numpy as np
import os
import pandas
import sys
import keras
import pytesseract
from matplotlib import pyplot as plt
from shutil import copyfile
from PIL import Image, ImageChops
import PIL.ExifTags
import imutils
from random import randint
from skimage import transform

Using TensorFlow backend.


In [3]:
def convert_images(filenames,directory,height,width,outdir):
    # Convert image to the same siz

    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Write it to file
        cv2.imwrite(outdir + "/" + file, resized_image)

    return images

In [4]:
def get_sizes(filenames,directory):
    # Read Images into a Numpy array
    
    # Get the array size
    array_size = len(filenames)
    height = np.array([])
    width = np.array([])
    
    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Get Size
        # Add it to numpy array
        height = np.append(height,image.shape[0])
        width = np.append(width,image.shape[1])
    
    return height,width

In [5]:
def find_rectangle(gray):
 
    ret,thresh = cv2.threshold(gray,100,255,1)
    _,contours,h = cv2.findContours(thresh,1,2)

    biggest_box = np.array([])

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    
        if len(approx)== 4:
            area = cv2.contourArea(cnt)
        
            if area > 1000 and area < 200000:
                biggest_box=cnt
        else:
            continue
            
    return biggest_box

In [6]:
def crop_bottom(image,box):

    x = np.array([])
    y = np.array([])
    for i in box:
        x = np.append(x,i[:,0])
        y = np.append(y,i[:,1])

    x1 = int(np.amin(x))
    y1 = int(np.amin(y))
    x2 = int(np.amin(x))
    y2 = int(np.amax(x))

    cropped = image[x1:y1, x2:y2]
    return cropped

In [7]:
def crop_text_bottomP(image,lines,width,height):
    #print("Probalistic Hough")
    cropped = np.array([])
    for x1,y1,x2,y2 in lines[0]:
        if y1 < int(height*.6) or y2 < int(height*.6):
            continue
        else:
            #print(x1,y1,x2,y2)
            cropped = image[0:y1,0:width]
            cropped = cropped[0:y2,0:width]
    return cropped

In [8]:
def crop_text_bottom(image,lines,width,height):
    #print("Regular Hough")
    cropped = np.array([])
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if y1 < int(height*.6) or y2 < int(height*.6):
            continue
        else:
            #cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 20)
            #print(x1,y1,x2,y2)
            cropped = image[0:y1,0:width]
            cropped = cropped[0:y2,0:width]

    return cropped

In [9]:
def process_text_image(filename,inputdir):
    
    image = cv2.imread(inputdir + "/" + filename, 0) # Working - Should Not Change

    height=image.shape[0]
    width=image.shape[1]

    # construct a sharpening filter
    kernel = np.array(([0, -1, 0],[-1, 5, -1],[0, -1, 0]), dtype="int")
    im1 = cv2.filter2D(image, -1, kernel)
    im2 = cv2.medianBlur(image, 3)

    edges1 = cv2.Canny(image,120,200,3)
    edges2 = cv2.Canny(im1,120,200,3)
    edges3 = cv2.Canny(im2,120,200,3)


    # Text needs to be cropped out.  0 Does not contain text, 1 has text
    has_text = 0
    text1 = pytesseract.image_to_string(edges1)
    text2 = pytesseract.image_to_string(edges2)


    if len(text1) > 5:
        has_text=1
    if len(text2) > 5:
        has_text=1
    
    cropped = np.array([])
        
    if has_text == 1:
        # Try Regular Houghlines
        maxLineGap = 1
        maxLineLength = width
        minLineLength = int(width*.6)
        threshold = 260
    
        #Try Regular Hough Lines
        if cropped.size == 0:
            lines1 = cv2.HoughLines(edges1, 1, np.pi/180, threshold)
            if lines1 is not None:
                cropped = crop_text_bottom(image,lines1,width,height)

        if cropped.size == 0:
            lines2 = cv2.HoughLines(edges2, 1, np.pi/180, threshold)
            if lines2 is not None:
                cropped = crop_text_bottom(image,lines2,width,height)
        
        if cropped.size == 0:
            lines3 = cv2.HoughLines(edges3, 1, np.pi/180, threshold)
            if lines3 is not None:
                cropped = crop_text_bottom(image,lines3,width,height)
    
        #Try Probalistic Hough Lines
        if cropped.size == 0:
            linesp1 = cv2.HoughLinesP(edges1,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp1 is not None:
                cropped = crop_text_bottomP(image,linesp1,width,height)
    
        if cropped.size == 0:
            linesp2 = cv2.HoughLinesP(edges2,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp2 is not None:
                cropped = crop_text_bottomP(image,linesp2,width,height)

        if cropped.size == 0:
            linesp3 = cv2.HoughLinesP(edges3,1,np.pi/180,100,minLineLength,maxLineGap)
            if linesp3 is not None:
                cropped = crop_text_bottomP(image,linesp3,width,height)
            
            
        if cropped.size != 0:
            return cropped,has_text
        
    return cropped,has_text

In [10]:
def crop_bottom_top(image):
    height=image.shape[0]
    width=image.shape[1]

    #print("Original Dimensions:", height,width)

    maxLineGap = 1
    maxLineLength = width
    minLineLength = int(height*.30)
    threshold=1000

    # Display Image
    #plt.imshow(image, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    edges1 = cv2.Canny(image,120,200,3)

    img_th = cv2.adaptiveThreshold(edges1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)

    # Display Image
    #plt.imshow(img_th, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    linesp1 = cv2.HoughLinesP(img_th,1,np.pi/180,threshold,minLineLength,maxLineGap)
    
    cropped = np.array([])

    if linesp1 is None:
        #print("No Lines found")
        return cropped
    else:
        max_height = int(height*.98)
        min_height = int(height*.02)

        previousmaxy=height
        previousminy=0

        # Crop Bottom Lines
        for line in linesp1:
            for x1,y1,x2,y2 in line:

                if y1 > max_height or y2 > max_height:
                    # We don't want diagonal or vertical lines
                    if y1 != y2:
                        continue
                    else:
                        if y1 < previousmaxy:
                            previousmaxy=y1
                            if cropped.size == 0:
                                cropped = image[0:y1, 0:width]
                            else:
                                height=cropped.shape[0]
                                width=cropped.shape[1]
                                #print(height,width)
                                cropped = cropped[0:y1, 0:width]

        # Crop top Lines
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if y1 < min_height or y2 < min_height:
                    # We don't want diagonal or vertical lines
                    if y1 != y2:
                        continue
                    else:
                        if y1 > previousminy:
                            previousminy=y1
                            if cropped.size == 0:
                                cropped = image[y1:height, 0:width]
                            else:
                                height=cropped.shape[0]
                                width=cropped.shape[1]
                                cropped = cropped[y1:height,0:width]
        return cropped

In [11]:
def crop_left_right(image):

    height=image.shape[0]
    width=image.shape[1]

    #print("Original Dimensions:", height,width)

    maxLineGap = 1
    maxLineLength = height
    minLineLength = int(height*.30)
    threshold=10
    
    edges1 = cv2.Canny(image,120,200,3)
    #plt.imshow(edges1, cmap = 'gray', interpolation = 'bicubic')
    #plt.show()

    linesp1 = cv2.HoughLinesP(edges1,1,np.pi/2,threshold=threshold,minLineLength=minLineLength,maxLineGap=maxLineGap)
    
    cropped = np.array([])

    if linesp1 is None:
        return cropped
    else:
        max_width = int(width*.95)
        min_width = int(width*.05)

        previousmaxx=width
        previousminx=0

        # Crop Right Frames
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if x1 > max_width or x2 > max_width:
                    # We don't want diagonal or horizontal lines
                    if x1 != x2:
                        continue
                    else:
                        if x1 < previousmaxx:
                            #print(previousmaxx)
                            #print("Cropping Right: ",x1,y1,x2,y2)
                            previousmaxx=x1
                                
        if previousmaxx != width:
            #print(previousmaxx)
            cropped = image[0:height, 0:previousmaxx]
            
        # Crop Left Frames
        for line in linesp1:
            for x1,y1,x2,y2 in line:
                if x1 < min_width or x2 < min_width:
                    # We don't want diagonal or horizontal lines
                    if x1 != x2:
                        continue
                    else:
                        if x1 > previousminx:
                            #print(previousminx)
                            #print("Cropping Left:",x1,y1,x2,y2)
                            previousminx=x1
                            
        if previousminx != 0: 
            #print(previousminx)
            if cropped.size == 0:
                cropped = image[0:height,previousminx:width]
            else:
                cropped = cropped[0:height,previousminx:width]
                                
    return cropped

In [12]:
def crop_tail(image):
    # Find the tail and crop everything else out
    height=image.shape[0]
    width=image.shape[1]

    dst =  cv2.fastNlMeansDenoising(image,None,10,7,21)
    edges1 = cv2.Canny(dst,100,250,3)

    _,contours, hierarchy = cv2.findContours(edges1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    cropped = np.array([])
    if not contours:
        print("No Contours")
    else:
        boxes = []
        for c in contours:
            if c.shape[0] < 100:
                continue
            else:
                (x, y, w, h) = cv2.boundingRect(c)
                boxes.append([x,y, x+w,y+h])

        boxes = np.asarray(boxes)
        if boxes.size != 0:
            left = np.min(boxes[:,0])
            top = np.min(boxes[:,1])
            right = np.max(boxes[:,2])
            bottom = np.max(boxes[:,3])
            cropped = image[top:bottom,left:right]
            
            new_width = cropped.shape[0]
            new_height = cropped.shape[1]
            area = new_width * new_height
            # Make sure they haven't cropped too much
            if area > 90000:
                return cropped
            else:
                return np.array([])
            
    return cropped

In [13]:
def lines_image(image):
    lines = cv2.Canny(image,120,200,3)

    return lines

In [14]:
def flip_image(image):
    horizontal_img = cv2.flip(image, 0 )
    vertical_img = cv2.flip(image, 1 )
    return vertical_img,horizontal_img

In [15]:
def add_noise(origdir,noisedir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    sigma = randint(20, 50)
    
    temp_image = np.float64(np.copy(image))
    h = temp_image.shape[0]
    w = temp_image.shape[1]
    noise = np.random.randn(h, w) * sigma

    noisy_image = np.zeros(temp_image.shape, np.float64)
    if len(temp_image.shape) == 2:
        noisy_image = temp_image + noise
    else:
        noisy_image[:,:,0] = temp_image[:,:,0] + noise
        noisy_image[:,:,1] = temp_image[:,:,1] + noise
        noisy_image[:,:,2] = temp_image[:,:,2] + noise
        
    cv2.imwrite(noisedir + "/" + i + "-" + file,noisy_image)

In [16]:
def random_rotate(origdir,noisedir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    angle =  randint(-10,10)
    rotated = imutils.rotate_bound(image, angle)
    cv2.imwrite(randomrotatedir + "/" + i + "-" + file,rotated)

In [17]:
def scale_in(origdir,scaleindir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    #randomized the scaling
    scale = random.uniform(0.1, 0.9)
    height, width, channels = image.shape
    scale_out = skimage.transform.rescale(image, scale=scale, mode='constant')
    #rescale Image after scale in
    resized_image = cv2.resize(scale_out, (width, height))
    cv2.imwrite(scaleindir + "/" + i + "-" + file,resized_image)

In [18]:
def scale_out(origdir,scaleoutdir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    #randomized the scaling
    scale = random.uniform(1.1, 2.2)
    height, width, channels = image.shape
    scale_out = skimage.transform.rescale(image, scale=scale, mode='constant')
    #rescale Image after scale in
    resized_image = cv2.resize(scale_out, (width, height))
    cv2.imwrite(scaleoutdir + "/" + i + "-" + file,resized_image)

In [19]:
def flip_image(origdir,scaleoutdir,file,i):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-19-e5b56dab401d>, line 2)

In [20]:
inputdir="/home/james/Kaggle/Whale/Data/train/1"
outputdir="/home/james/Kaggle/Whale/Data/train/2"
cropdir="/home/james/Kaggle/Whale/Data/train/3"
taildir="/home/james/Kaggle/Whale/Data/train/4"
labelsfile="/home/james/Kaggle/Whale/Data/train.csv"

In [21]:
labels = pandas.read_csv(labelsfile)



In [22]:
filenames = os.listdir(inputdir)
filenames.sort()





In [18]:
# Remove Large Text Boxes from bottom of picture.
# This is not perfect. But it takes alot of effort out of an otherwise manual process.
#or file in filenames:
#    new_image,has_text = process_text_image(file,inputdir)
#    if new_image.size != 0:
#        cv2.imwrite(outputdir + "/" + file,new_image)
#        print(file,": Cropping Image")
#    elif has_text==1:
#        # We still want to identify those with text that didn't get cropped
#        print(file,": Text, no crop")
#        copyfile(inputdir + "/" + file,outputdir + "/" + file )

In [19]:
# Get the average height and width of images so we can resize later
#height,width = get_sizes(filenames,inputdir)
#image_height = int(round(np.average(height)))
#image_width = int(round(np.average(width)))
# Manipulate the images and return the array
#chunk_images = convert_images(filenames,inputdir,image_height,image_width,outputdir)

In [20]:
# Remove Black lines from top and bottom, left and right
#for file in filenames:
#    image = cv2.imread(outputdir + "/" + file,0)
#    cropped = crop_bottom_top(image)

#    if cropped.size == 0:
#        cropped = crop_left_right(image)
#    else:
#        cropped = crop_left_right(cropped)
        
#    if cropped.size != 0:
#        cv2.imwrite(cropdir + "/" + file,cropped)
#        print(file,": Cropping Image")

In [21]:
# Find the Whales tail
#for file in filenames:
#    image = cv2.imread(cropdir + "/" + file,0)
#    cropped = crop_tail(image)

#    if cropped.size != 0:
#        cv2.imwrite(taildir + "/" + file,cropped)
#        print(file,": Cropping Image")

In [23]:
# Augment Data
augment_dir = "/home/james/Kaggle/Whale/Data/train/augment"
origdir = "/home/james/Kaggle/Whale/Data/train/augment/1"
rotatedir90 = "/home/james/Kaggle/Whale/Data/train/augment/2"
rotatedir180 = "/home/james/Kaggle/Whale/Data/train/augment/3"
rotatedir270 = "/home/james/Kaggle/Whale/Data/train/augment/4"
threshdir = "/home/james/Kaggle/Whale/Data/train/augment/5"
verticaldir = "/home/james/Kaggle/Whale/Data/train/augment/6"
horizontdir = "/home/james/Kaggle/Whale/Data/train/augment/7"
noisedir = "/home/james/Kaggle/Whale/Data/train/augment/8"
randomrotatedir = "/home/james/Kaggle/Whale/Data/train/augment/9"
combineddir = "/home/james/Kaggle/Whale/Data/train/augment/combined"

In [ ]:
iterations = 9
#Augment the data
for i in range(iterations):
    for filename in filenames:
        seed = randint(1,9)
        if seed == 1:
            # Just copy the image
            copyfile(origdir + "/" + filename, combineddir + "/" + i + "-"+ filename)
        if seed == 2:
            #Add Random Noise to image
            add_noise(origdir,combineddir,filename,i)
        if seed == 3:
            #Randomly rotate image
            random_rotate(origdir,combineddir,filename,i)
        if seed == 4:
            #Randomly rotate image
            scale_in(origdir,combineddir,filename,i)
        

In [24]:
full_exif = {}
file_csv = "/home/james/Kaggle/Whale/Data/train/exif.csv"
exif_count = 0
for file in filenames:
    img = Image.open("/home/james/Kaggle/Whale/Data/train/1/" + file)
    width, height = img.size
    try:
        exif = {
            PIL.ExifTags.TAGS[k]: v
            for k, v in img._getexif().items()
            if k in PIL.ExifTags.TAGS
        }
    except:
        full_exif[file] = {'width':width,'height':height}
        continue
        
    if len(exif) != 0:
        full_exif[file] = exif
        full_exif[file]['width'] = width
        full_exif[file]['height'] = height
        
        exif_count += 1
        #        #print("Exif: ",file)
    else:
        full_exif[file] = {'width':width,'height':height}
        #print("No Exif: ", file)
        continue
print(len(full_exif))
print("Exif Count: ",exif_count)    #

#Create list with all exif key names
key_list = []
for exif in full_exif:
    sub_dict = full_exif[exif]
    if len(sub_dict) != 0:
        for key in sub_dict:
            key_list.append(key)

# Add filename to list
key_list.insert(0, 'filename')

# Now instantiate a dataframe with those columns
df = pandas.DataFrame(columns=set(key_list))
    


# Now lets fill this dataframe with exif data
for exif in full_exif:
    sub_dict = full_exif[exif]
    
    if len(sub_dict) != 0:
        sub_dict['filename'] = exif
        df = df.append(sub_dict, ignore_index=True)
    else:
        sub_dict['filename'] = exif
        df = df.append(sub_dict, ignore_index=True)
    
#Set Filename as index
df = df.set_index('filename')
  
df.to_csv(file_csv, sep='\t')
# Write it to CSV file for training later
df.to_csv(file_csv, sep='\t')

25361
Exif Count:  7655


In [24]:
#for file in filenames:
#    #

#    sigma = randint(20, 50)
    
#    temp_image = np.float64(np.copy(image))

#    h = temp_image.shape[0]
#    w = temp_image.shape[1]
#    noise = np.random.randn(h, w) * sigma

#    noisy_image = np.zeros(temp_image.shape, np.float64)
#    if len(temp_image.shape) == 2:
#        noisy_image = temp_image + noise
#    else:
#        noisy_image[:,:,0] = temp_image[:,:,0] + noise
#        noisy_image[:,:,1] = temp_image[:,:,1] + noise
#        noisy_image[:,:,2] = temp_image[:,:,2] + noise
        
#    cv2.imwrite(noisedir + "/" + file,noisy_image)
#    print(file)

In [25]:
#Augment Data by randomly rotating images.
#for file in filenames:
#    angle =  randint(-10,10)
#    image = cv2.imread(origdir + "/" + file,0)
#    rotated = imutils.rotate_bound(image, angle)
#    cv2.imwrite(randomrotatedir + "/" + file,rotated)
#    print(file)

0000e88ab.jpg
0001f9222.jpg
00029d126.jpg
00050a15a.jpg
0005c1ef8.jpg
0006e997e.jpg
000a6daec.jpg
000f0f2bf.jpg
0016b897a.jpg
001c1ac5f.jpg
001cae55b.jpg
001d7450c.jpg
00200e115.jpg
00245a598.jpg
002b4615d.jpg
002f99f01.jpg
00355ff28.jpg
00357e37a.jpg
003795857.jpg
0041880bf.jpg
0043da555.jpg
00442c882.jpg
00464ff65.jpg
004775679.jpg
004ae9e26.jpg
004c0f43b.jpg
004e8ad5b.jpg
004f87702.jpg
0050ef29d.jpg
00514c876.jpg
0052ce2f5.jpg
00537ec91.jpg
00570db6b.jpg
005ce3100.jpg
00600ce17.jpg
006017ddf.jpg
0060f764a.jpg
006500b3d.jpg
006506edf.jpg
0067b3a20.jpg
006bdc6ff.jpg
00713060e.jpg
007218187.jpg
0073118da.jpg
007a7846f.jpg
007b62e05.jpg
007f3eb46.jpg
008124524.jpg
0084fe40b.jpg
0085064af.jpg
0085a6bd0.jpg
00869b59e.jpg
008bf89fc.jpg
008ebc50d.jpg
00910010d.jpg
00917baa4.jpg
0093454b6.jpg
009431af0.jpg
009589b45.jpg
0095fe89c.jpg
00961802f.jpg
009687166.jpg
0096c8ff3.jpg
009c45bc7.jpg
00a3dd76f.jpg
00a46c559.jpg
00a4de95e.jpg
00a70e649.jpg
00a9dc47d.jpg
00aae723d.jpg
00b7ffb8a.jpg
00b92e

05da28fcb.jpg
05dae8342.jpg
05db04583.jpg
05dd0ee6b.jpg
05dfa3773.jpg
05e5e0e89.jpg
05e81e346.jpg
05ea19d75.jpg
05ea2c861.jpg
05ed57e10.jpg
05ee04d0c.jpg
05ee1fc22.jpg
05eeabf1b.jpg
05f2f037c.jpg
05f71b26a.jpg
05fc2e35d.jpg
060bacbc7.jpg
060ffd336.jpg
0610203d1.jpg
0611099f3.jpg
06119661c.jpg
0613a4aa9.jpg
0616cbd68.jpg
06171a8a9.jpg
06172ce2c.jpg
0617ca20a.jpg
0617cbeed.jpg
0618f6c38.jpg
061996245.jpg
061f833ab.jpg
06209926d.jpg
06245ed98.jpg
062699bef.jpg
06276874e.jpg
0628d4679.jpg
062d0bd7e.jpg
06306504e.jpg
06307bbf6.jpg
0631db372.jpg
06336bf78.jpg
0633e8c04.jpg
063759573.jpg
06389a1fc.jpg
0638c9ae1.jpg
063d5700d.jpg
063e89413.jpg
064bafe62.jpg
064dd41a3.jpg
064de9c50.jpg
06504fa5b.jpg
0654b3e5b.jpg
065b72556.jpg
065fbc00f.jpg
066074afc.jpg
066228078.jpg
066c039dc.jpg
0670b3636.jpg
0679f42ec.jpg
067cdde9d.jpg
067d80626.jpg
068062d67.jpg
0680aab46.jpg
06822966b.jpg
06848f3e1.jpg
0689a1032.jpg
068f9cdba.jpg
06907cb5c.jpg
06948b93c.jpg
0695dc488.jpg
0696613e7.jpg
06988e2d8.jpg
069a1c

0ba2fe848.jpg
0ba3bb92d.jpg
0baa89964.jpg
0bb23359d.jpg
0bb469e7d.jpg
0bb5b9ae5.jpg
0bb67e44d.jpg
0bbf8f60a.jpg
0bc10f0f7.jpg
0bc2ff714.jpg
0bc4ab015.jpg
0bc500118.jpg
0bc8f5788.jpg
0bcb4b53a.jpg
0bcb9fc34.jpg
0bd1610f8.jpg
0bd64cea3.jpg
0bd65a44d.jpg
0bd7ff74f.jpg
0bda0c14f.jpg
0be63cacd.jpg
0be68a06b.jpg
0be897d0d.jpg
0bece9363.jpg
0bee24cfb.jpg
0bef01d4e.jpg
0bf303ed2.jpg
0bf8136cf.jpg
0bf9730f8.jpg
0bf980011.jpg
0c005aa88.jpg
0c037e179.jpg
0c057c0a6.jpg
0c076e252.jpg
0c08a4d6d.jpg
0c08f0cb6.jpg
0c0bf8fb8.jpg
0c0cea389.jpg
0c115fd9c.jpg
0c146080d.jpg
0c15936e3.jpg
0c166f137.jpg
0c18eda1c.jpg
0c1960aa4.jpg
0c19af6d4.jpg
0c1af7d67.jpg
0c2379d8a.jpg
0c25ed201.jpg
0c2653d9e.jpg
0c2adec98.jpg
0c30517d8.jpg
0c3197e04.jpg
0c341777d.jpg
0c36db7c9.jpg
0c3822f8c.jpg
0c39bd7ff.jpg
0c3afa7cd.jpg
0c3b7d753.jpg
0c401f637.jpg
0c44e92d7.jpg
0c45bf60f.jpg
0c4b0827e.jpg
0c4cf7f5d.jpg
0c508bef1.jpg
0c51473f9.jpg
0c51dffe2.jpg
0c5286449.jpg
0c5422934.jpg
0c5537f22.jpg
0c554ff65.jpg
0c5ce54ec.jpg
0c5fd5

11c985217.jpg
11cd20bbc.jpg
11ceb760d.jpg
11d39fc43.jpg
11d57cd9c.jpg
11df60816.jpg
11e29e5e2.jpg
11e370e11.jpg
11e3ea5f6.jpg
11e7f96c5.jpg
11e907470.jpg
11edd672b.jpg
11f476f56.jpg
11f591047.jpg
11f59c2b3.jpg
11f98ade5.jpg
11fc1d26c.jpg
1202f27b4.jpg
12035e558.jpg
12037f781.jpg
1204c90d2.jpg
12093d2aa.jpg
120bbc906.jpg
120f5245d.jpg
12126d255.jpg
121819c9a.jpg
12184a9dc.jpg
121880d7f.jpg
121a7c56b.jpg
121db0677.jpg
121e3acc2.jpg
121e713b1.jpg
1222fa72b.jpg
1226da218.jpg
122d0e026.jpg
122f497db.jpg
122fbab73.jpg
1232d5324.jpg
1235eb22c.jpg
1236b667c.jpg
1239463f4.jpg
123b794ba.jpg
1242405d7.jpg
124254cb7.jpg
1244ce135.jpg
124913ed3.jpg
124a0e40a.jpg
124b1ef2d.jpg
12509a8e1.jpg
12518411d.jpg
1255a88bc.jpg
125869ae7.jpg
125b7d50d.jpg
125ba1407.jpg
125c6370c.jpg
126002936.jpg
1262321e3.jpg
1262dba60.jpg
12670516b.jpg
1268ef193.jpg
126e44013.jpg
126fc51b0.jpg
1270df73e.jpg
1272df255.jpg
127309730.jpg
12768d93a.jpg
1277c7b44.jpg
127bdd2fe.jpg
127cf66b2.jpg
127d932ba.jpg
127ec7a42.jpg
128223

181ecc608.jpg
181fd95f4.jpg
182110c20.jpg
1822b7c68.jpg
18234111f.jpg
1825014de.jpg
18251a799.jpg
1829ac06d.jpg
182ac1bf2.jpg
182d5e290.jpg
18322eb9f.jpg
1837eedd8.jpg
183ae60da.jpg
18459a654.jpg
1845b343b.jpg
1847e87ee.jpg
184b85d7d.jpg
1850b6e39.jpg
18554c028.jpg
1855dac14.jpg
1860fb96e.jpg
1867e30c2.jpg
1868691a1.jpg
186dc4deb.jpg
1871a772e.jpg
1873d094a.jpg
18746ba21.jpg
1876ff23d.jpg
18780491e.jpg
187bb1839.jpg
1883237ff.jpg
188616175.jpg
1887a9835.jpg
188b04c45.jpg
188e5e66e.jpg
188f220f2.jpg
1891111cf.jpg
1892d17c2.jpg
1895c79c5.jpg
189a97d50.jpg
189b44a9d.jpg
189d5937b.jpg
189d85e90.jpg
18a26f162.jpg
18a278983.jpg
18a35d725.jpg
18a618c6b.jpg
18a6a8885.jpg
18a6dceea.jpg
18a73e2ee.jpg
18aae894c.jpg
18af303c2.jpg
18b2cfff5.jpg
18b36c8dc.jpg
18b39de07.jpg
18b7adcfa.jpg
18b9ebd3c.jpg
18bc422ca.jpg
18c3f55d6.jpg
18c7549fe.jpg
18c7f0c14.jpg
18ce3a307.jpg
18cf89a6f.jpg
18d2466bf.jpg
18d45e646.jpg
18d46c518.jpg
18d7cfb35.jpg
18d9e5c8b.jpg
18da3ddb5.jpg
18db030ba.jpg
18dd2427d.jpg
18e0d4

...

e256e3af7.jpg
e259a9f97.jpg
e25d6d9c3.jpg
e25dbc1e9.jpg
e25e61a20.jpg
e261c161a.jpg
e261fb7ae.jpg
e26658de8.jpg
e26d1cbc8.jpg
e271b430b.jpg
e276017b3.jpg
e278c4b2a.jpg
e27c1ff6e.jpg
e27ef0ae3.jpg
e27fa52d9.jpg
e2865b4ed.jpg
e288d66cf.jpg
e28af5a44.jpg
e28bd69e9.jpg
e28d79b1c.jpg
e294aedec.jpg
e296b7567.jpg
e29a9d160.jpg
e29fc82ff.jpg
e2a0a5cc5.jpg
e2a248061.jpg
e2a8a194c.jpg
e2ac4810b.jpg
e2adac946.jpg
e2aeb6004.jpg
e2aec85b7.jpg
e2b2cdb8b.jpg
e2b4227db.jpg
e2b438e6a.jpg
e2b621c52.jpg
e2b6b121f.jpg
e2b6d6773.jpg
e2b9e9f5c.jpg
e2bc464c2.jpg
e2c0d3bfe.jpg
e2c26c51d.jpg
e2c2ee9eb.jpg
e2ca84843.jpg
e2cd8774b.jpg
e2ce362b2.jpg
e2ced2cf6.jpg
e2d079d5d.jpg
e2d18e457.jpg
e2d1bc6e6.jpg
e2d435827.jpg
e2d4aa723.jpg
e2d8ed549.jpg
e2da18134.jpg
e2daae0fc.jpg
e2dd4ae11.jpg
e2ddb23b2.jpg
e2deae874.jpg
e2e1cb448.jpg
e2e3c5ce2.jpg
e2e699b4e.jpg
e2e74ce81.jpg
e2ebc54f6.jpg
e2ec00d35.jpg
e2f1b6c4a.jpg
e2f802201.jpg
e2f9b3fd1.jpg
e3003a878.jpg
e30223a97.jpg
e302cf945.jpg
e306f6b62.jpg
e3079e104.jpg
e30abf

e8a37e2e0.jpg
e8a547e25.jpg
e8ac232ce.jpg
e8adedbac.jpg
e8af3a144.jpg
e8b19f1d5.jpg
e8b1cfcd1.jpg
e8b971b90.jpg
e8c13b4a3.jpg
e8c1f45a2.jpg
e8c52025f.jpg
e8c652384.jpg
e8c76f24c.jpg
e8ca0c506.jpg
e8ca1a08c.jpg
e8d1379ff.jpg
e8d3aa93b.jpg
e8d54b2dd.jpg
e8d78f3ce.jpg
e8d930522.jpg
e8d9d575d.jpg
e8df40ead.jpg
e8e004639.jpg
e8e011bbe.jpg
e8e615309.jpg
e8e7bd363.jpg
e8e992bdf.jpg
e8f54be4a.jpg
e8f67de5b.jpg
e8f730067.jpg
e8f98cf7a.jpg
e8fa31914.jpg
e8fc70a43.jpg
e9007353e.jpg
e90138d00.jpg
e90213c2b.jpg
e90ad944e.jpg
e90d79193.jpg
e911264d0.jpg
e9121803d.jpg
e9146b5b1.jpg
e9164450d.jpg
e91a86909.jpg
e91af7268.jpg
e91b1e52f.jpg
e91b93c2d.jpg
e91d88ec3.jpg
e91deea5c.jpg
e91f67b7b.jpg
e9240f133.jpg
e92422386.jpg
e9280c18c.jpg
e929802ba.jpg
e92c45748.jpg
e92d449c5.jpg
e9300b69b.jpg
e930f7403.jpg
e932b9c13.jpg
e933d91f1.jpg
e9340e338.jpg
e934a2348.jpg
e934ced7c.jpg
e93567ffa.jpg
e93cca68d.jpg
e93ebc16b.jpg
e93f85544.jpg
e940131cc.jpg
e940d590d.jpg
e943c987c.jpg
e943fdaa5.jpg
e94403106.jpg
e94476

ee6591720.jpg
ee691759d.jpg
ee6d38970.jpg
ee6e25bf4.jpg
ee6e9aca9.jpg
ee6ffad89.jpg
ee7653a0e.jpg
ee783803c.jpg
ee7a783be.jpg
ee817ee55.jpg
ee8268db7.jpg
ee8278f2a.jpg
ee87a2369.jpg
ee8868b71.jpg
ee88bb6d0.jpg
ee896cff2.jpg
ee8a6165d.jpg
ee8f5e2f7.jpg
ee9156433.jpg
ee9237f9f.jpg
ee935a361.jpg
ee93e741d.jpg
ee949ebcd.jpg
ee955ddd7.jpg
ee96b5be6.jpg
ee96db7a4.jpg
ee973fb51.jpg
ee9d31802.jpg
ee9f1013f.jpg
eea000ebc.jpg
eea182824.jpg
eea317b96.jpg
eea64152a.jpg
eea6ae9bc.jpg
eeaa9a20f.jpg
eeacca43a.jpg
eeb35fd57.jpg
eeb417d9e.jpg
eeb49a892.jpg
eeba53ac1.jpg
eebafcb82.jpg
eebbb2e0c.jpg
eebd9d2f6.jpg
eebff9541.jpg
eec0652fe.jpg
eec2ba20e.jpg
eec58304b.jpg
eec64c659.jpg
eecdab581.jpg
eed024c1b.jpg
eed4a40e7.jpg
eed85338a.jpg
eedc1cbee.jpg
eeddcd5ce.jpg
eee15e60a.jpg
eee40f386.jpg
eeea46f35.jpg
eeef839d1.jpg
eef0f480f.jpg
eef2d76ce.jpg
eef7017b5.jpg
eef72465c.jpg
eef8b4ef9.jpg
ef030b1e4.jpg
ef082d2bc.jpg
ef129445d.jpg
ef135a3bc.jpg
ef179c103.jpg
ef17a3b8e.jpg
ef18a20ed.jpg
ef1adb68a.jpg
ef1d5f

f44173db8.jpg
f4479d9bc.jpg
f44c6c117.jpg
f44e328c4.jpg
f45033a67.jpg
f45061aff.jpg
f4525679c.jpg
f45673e8a.jpg
f456de032.jpg
f45ba20c2.jpg
f46117c0d.jpg
f461266f6.jpg
f4628b9de.jpg
f462b28a9.jpg
f463b8263.jpg
f464c1b66.jpg
f4654f157.jpg
f46f2186f.jpg
f46ff1e8a.jpg
f4717d591.jpg
f471f175e.jpg
f47adf5a4.jpg
f480f9980.jpg
f4827e52f.jpg
f483843b5.jpg
f483b0c75.jpg
f489241d9.jpg
f48a4940f.jpg
f48e3a5c0.jpg
f491dfc21.jpg
f4921d87d.jpg
f492355a9.jpg
f49347c85.jpg
f49669cdd.jpg
f496960d0.jpg
f496c5a75.jpg
f49897dde.jpg
f499e93b0.jpg
f49d137b0.jpg
f49e3801f.jpg
f4a76159e.jpg
f4a8505d2.jpg
f4a8a21a7.jpg
f4add4879.jpg
f4af7d347.jpg
f4b3b10f6.jpg
f4b478605.jpg
f4b5a4037.jpg
f4b5c9a83.jpg
f4b6cd07e.jpg
f4b6fc1b7.jpg
f4b8f1aed.jpg
f4bcadaa2.jpg
f4c082f0c.jpg
f4c555700.jpg
f4c6f8500.jpg
f4c7a2b75.jpg
f4cb4d05b.jpg
f4d563720.jpg
f4d793ab7.jpg
f4dccdeae.jpg
f4e19c4fa.jpg
f4e4e0e35.jpg
f4e551f67.jpg
f4e634968.jpg
f4e9f8d1c.jpg
f4ea4b09f.jpg
f4eabaf0e.jpg
f4ef38e12.jpg
f4f53c15b.jpg
f4f6dd2bf.jpg
f4fa5b

fa43c52f0.jpg
fa4ddcece.jpg
fa4f28cfc.jpg
fa5211df9.jpg
fa5327441.jpg
fa54e0c0f.jpg
fa559474f.jpg
fa5851a8a.jpg
fa5a0302b.jpg
fa5c50f89.jpg
fa5eec963.jpg
fa5f6534b.jpg
fa620ca00.jpg
fa6692809.jpg
fa6be3f69.jpg
fa6ee3eea.jpg
fa73bc2ab.jpg
fa74acdd2.jpg
fa773d1c8.jpg
fa7ad2a07.jpg
fa7d1b5f2.jpg
fa7fe2e4f.jpg
fa810fab3.jpg
fa844738c.jpg
fa8868884.jpg
fa888c2fe.jpg
fa8bb6306.jpg
fa944443c.jpg
fa9583ab2.jpg
fa95c39b4.jpg
fa9699838.jpg
fa9845bb1.jpg
fa985b64b.jpg
faa21a9e0.jpg
faa32ec3e.jpg
faa6859de.jpg
faad2501c.jpg
faafebb72.jpg
fab45c48b.jpg
fab732129.jpg
fab859ea1.jpg
fabc19a85.jpg
fabd88ee8.jpg
fac003725.jpg
fac110462.jpg
fac859a72.jpg
fad568d6b.jpg
fad6a739c.jpg
fad723d91.jpg
fad7fd94b.jpg
fad97f8c2.jpg
fadcbb6fc.jpg
fae3fc649.jpg
fae67b810.jpg
fae8fb4ce.jpg
faea123b0.jpg
faec9335c.jpg
faef1e205.jpg
faef8c481.jpg
faf1ae75f.jpg
faf42b9c6.jpg
faf7a3cad.jpg
faf949a01.jpg
fafb30b7e.jpg
fb03d93b4.jpg
fb0445bc3.jpg
fb0971819.jpg
fb0d680fd.jpg
fb119ca04.jpg
fb11c4396.jpg
fb140350a.jpg
fb1632

In [28]:
#Augment Data by Adding Lines
#for file in filenames:
#    image = cv2.imread(origdir + "/" + file,0)
#    threshimage = lines_image(image)
#    cv2.imwrite(threshdir + "/" + file, threshimage)


In [29]:
#Augment Data by Flipping Image
#for file in filenames:
#    image = cv2.imread(origdir + "/" + file,1)
#    vertimage,horimage = flip_image(image)
#    
#    cv2.imwrite(verticaldir + "/" + file, vertimage)
#    cv2.imwrite(horizontdir + "/" + file, horimage)

